In [20]:
import requests
import csv
import time
import pandas as pd
import os
from datetime import datetime

# 设置程序运行的总时间为 15 秒（为了方便测试）
total_running_time = 15

# 获取程序开始运行的时间
start_time = time.time()

# 创建 sorted 文件夹用于保存排序后的文件
if not os.path.exists('./sorted'):
    os.mkdir('./sorted')

# 创建 bithumb_orderbook.csv 文件并写入列名
header = 'price   |quantity|Type|timestamp'
with open('./all/bithumb_orderbook.csv', mode='w', newline='') as f:
    f.write(header + '\n')

while True:
    # 判断程序是否已经运行了指定的时间
    if time.time() - start_time > total_running_time:
        break
        
    response = requests.get('https://api.bithumb.com/public/orderbook/BTC_KRW/?count=5')
    book = response.json()
    data = book['data']
    
    # 将asks和bids的价格, asks和bids的数量作为两列，并新建Type列区分asks和bids
    bids = pd.DataFrame(data['bids'], columns=['price', 'quantity'])
    bids['Type'] = 0  # 添加 Type 列并设置值为 0，代表 bids
    asks = pd.DataFrame(data['asks'], columns=['price', 'quantity'])
    asks['Type'] = 1  # 添加 Type 列并设置值为 1，代表 asks
    
    # 使用 merge 合并 asks 和 bids
    df = pd.merge(bids, asks, how='outer')
    
    # 创建包含当前时间的 DataFrame，并将 timestamp 值的前五位添加到时间后面
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    time_df = pd.DataFrame({'time': [now] * len(df)})
    
    # 合并时间和 asks/bids 数据的 DataFrame
    df = pd.concat([df, time_df], axis=1)
    
     # 格式化输出并保存到 CSV 文件中
    df.to_csv('./all/bithumb_orderbook.csv', index=False, header=False, mode='a', sep='|')


    time.sleep(1)
